In [0]:
import pyspark
from pyspark.sql.functions import col, concat, lit, regexp_replace, when, length, lpad, to_timestamp, max, rank
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import airporttime
from datetime import datetime, timedelta

In [0]:
blob_container = "w261-scrr" # The name of your container created in https://portal.azure.com
storage_account = "midsw261rv" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261scrr" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261scrrkey" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)


In [0]:
%run "./libs/weather_data"

In [0]:
%run "./libs/station_data"

In [0]:
%run "./libs/airlines_data"

In [0]:
%run "./libs/airport_codes_data"

In [0]:
%run "./libs/data_joins"

In [0]:
airport_codes = get_airport_codes_full()

In [0]:
df_weather_full = get_full_weather_data().filter(col('DATE') > "2019-01-01T00:00:00.000").cache()
#df_airlines_train = get_full_airlines_data_train()
df_airlines_test = get_full_airlines_data_test()
df_stations = get_station_data()

In [0]:
#remove airlines that are not mapped to weather data
airline_codes = df_airlines_test.select('ORIGIN').distinct() #366 distinct codes
iata_join = airline_codes.join(airport_codes, airline_codes.ORIGIN == airport_codes.iata_code, how = 'left').cache() #mapping airlines to iata codes to stations
temp = iata_join.join(df_stations.select('neighbor_name', 'neighbor_state', 'neighbor_call'), df_stations.neighbor_call == iata_join.ident, how = 'left').cache()
null_airline = temp.filter(temp.neighbor_call.isNull()).select('ORIGIN')
display(null_airline)

ORIGIN
PIR
BFM
PAE
XWA
ATY
BKG
PSE
SPN
OGS
GUM


In [0]:
# print(df_airlines_train.count()) #
df_airlines_test_full = df_airlines_test.filter(~df_airlines_test.ORIGIN.isin(null_airline.rdd.flatMap(lambda x: x).collect()))
#df_airlines_test_full = df_airlines_test.filter(~df_airlines_test.ORIGIN.isin(null_airline.rdd.flatMap(lambda x: x).collect()))
# print(df_airlines_train_full.count()) #
# display(df_airlines_train_full)

In [0]:
df_flight_stamps_full = flight_timestamps(df_airlines_test_full)

In [0]:
# df_airlines_train.count() #24,324,804

In [0]:
df_stations_ranked = rank_closest_stations(df_stations, n=3)

In [0]:
df_airport_to_weather = airport_to_weather(airport_codes, df_stations_ranked, df_weather_full)

In [0]:
df_join = full_join(df_flight_stamps_full, df_airport_to_weather)

In [0]:
df_join.write.parquet(f"{blob_url}/test_full_join_0404")